<a href="https://colab.research.google.com/github/franzis17/EnronEmailAnalysis/blob/main/Business_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b>Introduction</b>

The Enron Corporation is a company based in America that provides energy, services, and commodity. Due to fraudulent accounting practices and financial scandals, Enron is notorious for corporate fraud and was exposed in 2001.

This report aims to discover potential topics of interest and patterns of communication, in relation to the fraudulent activities that Enron did, by analysing the volume of email over time, Top 20 email senders and email receivers, and analysing email subject keywords.

## <b>Assessment Objective</b>
The objective of this assessment is to understand SQLite databases, data exploration, data visualisation, and to apply industry best practices in programming by analysing the Enron Email dataset using Python in Google Colab notebooks.

# <b>Program Initialisation</b>

<b>IMPORTANT:</b> Must do the following before running any analysis

## <b>Source of data</b>

Upload the Database/Dataset.<br>
<b>Note:</b> Must select 1 option out of the 3 options of obtaining the Dataset
1. Download the Full/Complete Dataset by running the first code below and uploading to the Notebook
2. Download the Extract/Partial Dataset by running the second code below and uploading to the Notebook
3. Mount Google Drive: Once the Full dataset has been downloaded, upload the Dataset to your Google Drive and connect to your Google Drive instead of repeatedly uploading the Full/Complete Dataset.

In [ ]:
# Download the Complete Enron Email Database
!wget -O enron.db https://curtin-my.sharepoint.com/:u:/g/personal/211934g_curtin_edu_au/EaYagsqa2r1Bi5wtHbswGFwBH2kd2uTnz6rlka7GI36GUQ?dow

In [ ]:
# Download the Extract of Enron Email Database
!wget -O extract.db https://curtin-my.sharepoint.com/:u:/g/personal/211934g_curtin_edu_au/EbNeFqkc_g1EmQZzAKqc_w8BYeUTUVyexiSsmDbZb1EqAQ?download=1

In [ ]:
# If database is stored in your Google Drive
from google.colab import drive
drive.mount('/content/drive')

## <b>Initialise all packages needed in the program</b>
<b>Note:</b> The Development notebook contains the purposes of why each packages were used.



In [ ]:
import calendar
import matplotlib.pyplot as plt
import pandas as pd
import re  # regular expression
import sqlite3
from wordcloud import WordCloud

# Generate Stopwords - Obtained from Ref[2]
from google.colab import output
!curl -Ol https://raw.githubusercontent.com/michael-borck/isys2001-worksheets/main/stopwords.py
output.clear()
print("Required packages installed")

## <b>Connect to the database</b>

<b>Note:</b> Must adjust the path of the database according to where the database is located

In [ ]:
# Copy-and-Paste the file path of the database here
path_to_db = '/content/drive/MyDrive/a-quick_uploads/enron.db'

# Connect to the database using 'sqlite3'
conn = sqlite3.connect(path_to_db)

# Create a cursor to navigate each rows in the database and query any specific data from the dataset
cur = conn.cursor()

## <b>Functions</b>

* MUST run the following functions before any analysis.

In [ ]:
def query_db(table_name):
    sql='''
    SELECT * FROM {}
    '''.format(table_name)
    return pd.read_sql(sql, conn)

def query_db_for_messages():
    return query_db('message')

def query_db_for_employees():
    return query_db('employeelist')

def query_db_for_recipients():
    return query_db('recipientinfo')

# <u>**Analysis 1.**</u> Email Traffic Over Time

Aim: Analyze the volume of emails sent over time by counting the number of messages of each employees sent per month.

## <b>Figure 1.1: Total number of emails sent per year</b>

In [ ]:
'''
Note:
> Lines 13 to 20 were generated by ChatGPT. More info on Ref.ChatGPT[1]
'''

messages_df = query_db_for_messages()

# Make a new year table to sort the total number of messages by year
messages_df['date'] = pd.to_datetime(messages_df['date'], errors='coerce')
messages_df['year'] = messages_df['date'].dt.year
messages_per_year = messages_df.groupby('year').size()

# Plot the total number of messages per year
plt.figure(figsize=(15,5))
messages_per_year.plot(kind='line', marker='o')
plt.xticks(messages_per_year.index, rotation=90)
plt.xlabel('Year')
plt.ylabel('Number of messages per year')
plt.title('Total Number of Messages Per Year')
plt.show()

## <b>Figure 1.2: Total number of emails sent per month in the year 2001</b>

In [ ]:
'''
Note:
  > Lines 8 to 29 were generated by ChatGPT. More info on Ref.ChatGPT[3]
'''

messages_df = query_db_for_messages()

date_df = pd.DataFrame()
year_2001_df = pd.DataFrame()

messages_df['date'] = pd.to_datetime(messages_df['date'], errors='coerce')

year_2001_df = messages_df[(messages_df['date'].dt.year == 2001)]

messages_per_month = year_2001_df.groupby(year_2001_df['date'].dt.month).size()

# Get the month names corresponding to the month numbers
month_names = [calendar.month_name[month_num] for month_num in messages_per_month.index]

# Create a line chart showing the messages per month
messages_per_month.plot(kind='line', marker='o')
plt.title('Number of Messages in Each Month in 2001')
plt.xlabel('Month')
plt.ylabel('Number of Messages')

# Set the x-axis tick labels as the month names
plt.xticks(messages_per_month.index, month_names, rotation='vertical')

plt.show()

## <b>Discussion</b>

According to Investopedia (Ref[3]), Enron was exposed for its fraudulent accounting practices in the year 2001 and filed for bankruptcy in December of 2001. Additionally, the outputs shown on figure 1.1 displays that 2001 had the most amount of email communications out of all the other years and as shown in figure 1.2, the months before Enron filed bankruptcy had the most traffic out of the other months in the year 2001. This evidence suggests that the employees of Enron were pressured from the financial loss Enron was experiencing at the time and had to communicate to stakeholders of Enron via email to handle any issues regarding Enron’s downfall.

# <u>**Analysis 2.**</u> Top Senders and Receivers
Aim: Identify the most frequent email senders and recipients by aggregating(collecting) the data in the 'Message' and 'RecepientInfo' tables.

## <b>Top 10 Email <i>Senders</i></b>

In [ ]:
'''
Lines 9 to 24 are generated by ChatGPT: more info on Ref.ChatGPT[2]
'''

# Obtain all messages and employees from the database
messages_df = query_db_for_messages()
employees_df = query_db_for_employees()

# Merges the datasets 'messages' and 'employees' to count which messages are whose
merged_df = messages_df.merge(employees_df[['Email_id', 'firstName', 'lastName']], left_on='sender', right_on='Email_id', how='left')

# Get each employees full name by combining their firstName and lastName
merged_df['fullName'] = merged_df['firstName'] + ' ' + merged_df['lastName']

# Count the number of messages based on the amount of each unique email addresses
employee_counts = merged_df['fullName'].value_counts()
top_employees = employee_counts.head(10)

# Plot to Bar Chart the employees who have the highest email sent
top_employees.sort_values(ascending=True).plot(kind='barh')
plt.xlabel('Number of emails')
plt.ylabel('Employee Name')
plt.title('Top 10 Email Senders')
plt.show()

## <b>Top 10 Email <i>Receivers</i></b>

In [ ]:
'''
Lines 10 to 25 are copied from "Top 10 Email Senders" and changed to merge
recipients and employees dataset instead of messages and employees
'''

# Obtain all recipients and employees from the database
recipients_df = query_db_for_recipients()
employees_df = query_db_for_employees()

# Merges the datasets 'messages' and 'recipientinfo' to count how many emails are received by each employees
merged_df = recipients_df.merge(employees_df[['Email_id', 'firstName', 'lastName']], left_on='rvalue', right_on='Email_id', how='left')

# Get each employees full name by combining their firstName and lastName
merged_df['fullName'] = merged_df['firstName'] + ' ' + merged_df['lastName']

# Count the number of messages based on the amount of each unique email addresses
employee_counts = merged_df['fullName'].value_counts()
top_employees = employee_counts.head(10)

# Plot to Bar Chart the employees who have the highest email sent
top_employees.sort_values(ascending=True).plot(kind='barh')
plt.xlabel('Number of emails')
plt.ylabel('Employee Name')
plt.title('Top 10 Email Receivers')
plt.show()

## <b>Discussion</b>

Based on the analysis of top email senders, Jeff Dasovich, Vince Kaminski, Kay Mann, Sara Shackleton, Tana Jones, Chris Germany, Matthew Lenhart, Debra Pelingiere, Steven Kean, and Gerald Nemec were 10 Enron employees who have sent the most emails. Relatively, the top 10 email receivers were Jeff Dasovich, Richard Shapiro, Steven Kean, James Steffes, Tana Jones, Sara Shackleton, Mark Taylor, Louise Kitchen, Vince Kaminski, and Mark Guzman. Jeff Dasovich, Vince Kaminski, Tana Jones, Sara Shackleton were the most common names that were found.

For the purposes of this assignment. Only the most common names from both the most senders and receivers of emails are studied. Note that the following information about each person was obtained from the dataset (Enron.db) given in the assignment and the program “DB Browser for SQLite” was used to explore messages contained in the email.
<br><br>
The following are more information about the common people found in both top senders and receivers of email:
<br><br>
<b>Jeff Dasovich</b>
<br>
The person who sent the most emails is Jeff Dasovich. Jeff Dasovich was Enron’s governmental affairs executive (Linkurious, Ref-4). The earliest email Dasovich sent was in November of 1999 and was sent to Charry Lising. The message contained information about a cruise that is completely unrelated to Enron. The oldest email Dasovich sent was in January 8, 2002. The email contains a message that was sent by Allen DeBrum and Dasovich forwarded the email to 5 individuals named Steven Harris, Michelle Lokay, Stephanie Miller, Jane Tholt, and James Steffes. The message talks about the objective of California Gas Transmission Pacific Gas and Electric Company (CGTPGEC), which was communicated by Allen DeBrum. DeBrum was the Energy Trading Representative of CGTPGEC.
<br><br>
<b>Vince Kaminski</b>
<br>
The 2nd person who sent the most emails is Vince Kaminski. Vince Kaminski was a managing director for Enron, who testified that he disagreed with Enron’s fraudulent activities. Alexei (NYTimes, [Ref-5]) states that Vince Kaminski was Enron’s managing director. Alexei added that Mr. Kaminski dissented to the formation of off-balance-sheet partnerships directed by Mr. Fastow, who was the CFO of Enron. Additionally, Brian (Ref-6) states that Enron hid billions of dollars in losses from failed projects and deals, by using complicated off-balance sheet entities and SPEs. The evidence suggests that Kaminski was not a part of Enron’s fraudulent activities. In summary, the former managing director, Mr. Kaminski wanted to prevent Enron from doing acts of fraudulence.
<br><br>
<b>Tana Jones</b>
<br>
More information about Tana Jones is obtained by studying the earliest and latest emails that she sent. The earliest email she sent was in May of 1999 and was sent to an individual named Mark Elliott. The message is considered to be a personal message completely unrelated to Enron as it mentions going to a resort. The oldest email Tana sent was in February of 2002, which was after Enron filed for Bankruptcy. The email was sent to 2 individuals named Sara Shackleton and Robert Bruce. The message contained information about whether Tana and the aforementioned 2 individuals should withdraw from trading on the NYMEX. According to James (Ref[7]), NYMEX is a commodities trading exchange. The message implies that Tana is a commodities trader for Enron.
<br><br>
<b>Sara Shackleton</b>
<br>
Sara Shackleton’s earliest and oldest emails are analysed to gain an in-depth understanding of her role in Enron. The earliest email she sent was in May of 1999 and was sent to Patricia Cini. The email contains a message where Sarah asks Patricia if she has sent their form of guarantee, which is a form used for transactions to guarantee an outcome given a condition. The oldest email she sent was in March of 2002 and was sent to Neil Kelly. Based on the contents of the message, Neil and Sara are discussing a global settlement with TXU. As stated on Wikipedia (Ref[8]), TXU is an American retail electricity provider. The message suggests that TXU and Enron had a legal dispute with each other, potentially emerging from Enron’s bankruptcy, and came upon a legal agreement.

# <u>**Analysis 3.**</u> Subject Keyword Analysis
Aim: Extract keywords from email subjects in the 'Message' table and analyze the frequency of words used to understand common topics of discussion.

In [ ]:
from stopwords import ENGLISH_STOP_WORDS

messages_df = query_db_for_messages()

# Obtained from Curtin Tutorial Ref[2]
def clean(text):
    ''' Uses regular expresison to extract english letter and digits from the supplied text. '''
    regExp = "(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"
    return ' '.join(re.sub(regExp, " ", text).split())

# Create a new column as a placeholder for the clean subject keywords
messages_df['Clean Subject'] = messages_df['subject'].apply(clean)

# Add additional words to stopwords that do not have much meaning
additional_stop_words = {'New', 'Meeting', 'Update', 'Enron', 'Market', 'Power',
    'Caiso', 'Notice', 'Hour', 'HourAhead', 'California', 'FW', 'Date', 'Fwd',
    'Letter', 'Deal', 'Draft', 'Report', 'Contract', 'Conference', 'Start',
    'Codesite', 'PG E', 'Agreement', 'Request', 'Issue', 'Change', 'PG', 'Revised',
    'Presentation', 'Today', 'Question', 'Status', 'EOL', 'Mention', 'Comment',
    'Employee', 'E', 'Confirmation', 'Project', 'Reminder'}
ENGLISH_STOP_WORDS.update(additional_stop_words)

# Create and Display the wordcloud
subjects = ' '.join(messages_df['Clean Subject'])
wordcloud = WordCloud(width=680, height=480, margin=0, stopwords=ENGLISH_STOP_WORDS).generate(subjects)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.margins(x=0, y=0)
plt.show()

## **Discussion:**

Analysis 3 is conducted above to find any keywords in the subjects of all emails to gain an understanding about the fraudulent activities of Enron. Based on the outputs of analysis 3, Ken Lay was one of the main topics discussed in the emails. Below provides more information about the topic.

<br>

### <b>Ken Lay</b> - Employees demanding Ken Lay to donate
Ken Lay is an important subject regarding Enron's unethical business practices as Ken Lay was the CEO and Founder of Enron, which potentially indicates that Ken could have some degree of involvement with Enron's immoral accounting activities and financial scandal. 

Using the DB Browser for SQLite program and filtering the message.subject table with "Ken Lay" as a search word, there was approximately a 1000 emails sent to Mr. Ken Lay, which all stated the same subject "Demand Ken Lay Donate Proceeds from Enron Stock Sales". The content of the emails talks about demanding Ken to donate money to Enron Employee Transition and REACH funds. Additionally, the email states that the "New York Times reported that Ken Lay sold $101 million worth of Enron stock while aggressively urging the company's employees to keep buying it". Assuming the New York Times' report is true, then this means Ken Lay acted in an unethical manner, thereby making him one of the people responsible for the illicit act.

Given the messages of Enron's employees to Ken Lay, Ken Lay is one of the accused people to be involved in Enron's unethical accounting practices.

## Close the connection to the database
**Note:** Use when done with the database

In [ ]:
conn.close()

# Interpretation and Conclusion

## Summary

This project provides an insight into Enron’s communication patterns and discovers potential topics of interest by conducting three analyses, analysis of email traffic over time, identification of the top email senders and receivers, and analysis of the most frequently used words.

Summary of the analysis:
* Upon analysing the volume of emails throughout time, it is clear that Enron email communication had more activity in the year of 2001 due to the downfall of the company.
* Based on the second analysis of identifying the top senders and receivers of email, most employees have little evidence of their involvement in the fraudulent practices attributed to Enron. 
* By filtering out meaningless words from the subject attribute of each email, the third analysis has identified that most email subjects had the keyword “Ken Lay”. Using the DB Browser for SQLite program and filtering the message.subject with the word “Ken Lay”, most of the emails are about hundreds of employees demanding Ken Lay to donate fraudulently obtained money to Enron Employee Transition Fund.

By performing the three analysis described previously, SQLite databases were clearly understood and extracted the data from the database effectively, used the program “DB Browser for SQLite” to explore the data that is outputted from the analysis, plotted the results of the analysis in the appropriate graph specified in the specification to provide visualisation of the data, and followed the PEP8 programming style naming conventions to apply best practices in programming.


## Limitations

The assignment expects to choose 3 methods of analysis and the 3 analyses provided in the specification only takes an introductory level understanding about the communication patterns of the Enron Email dataset. Based on these expectations, only a limited level of information can be understood about Enron’s communication patterns. For example, Analysis 2 is about finding the top senders and receivers of email, but the output does not provide an in-depth understanding of who the top senders and receivers are and their relation to the problem of Enron’s fraudulent activities.

Analysis 2 can be improved by gaining more understanding about the relationships between employees based on who communicates with who according to the email dataset.


## Reflection on the Usability and Effectiveness of Python Notebooks

A part of the reason why Python Notebooks are usable is its cross-platform compatibility, which allows different users on different machines to access the application independent of which machine they are using. In my experience, I use Google Colab on a Curtin University machine, which runs on Linux and when I’m at home, I run it on a Windows Machine.

When it comes to Data Science projects, Python Notebooks are remarkably useful as they allow users to document the methods and results of their analysis seamlessly. As seen in this Python Notebook, I was able to run programmatic analytical methods and provide my insights about the results of the analysis. Compared to traditionally running Python scripts on a local machine to output a result or a graph, using a Python Notebook makes it less complicated for a client to run a Python script themselves and adds trust since the code can be seen and the output is produced easily.


# <b>Reference List</b>

## <b>References</b>

* [1] https://pandas.pydata.org/docs/user_guide/10min.html#plotting
* [2] Obtained from WK-9 tutorial of Unit ISYS2001-IBP from Curtin University
* [3] Investopedia. https://www.investopedia.com/updates/enron-scandal-summary/#:~:text=However%2C%20the%20entire%20edifice%20was,the%20world%20at%20that%20time
* [4] Linkurious. About Jeff Dasovich. https://linkurious.com/blog/investigating-the-enron-email-dataset/
* [5] NYTimes. About Vince Kaminski. https://www.nytimes.com/2006/03/15/business/businessspecial3/exenron-officer-says-he-warned-of-shady.html
* [6] Brian. About Enron Executives. https://www.investopedia.com/enron-executives-6831970
* [7] James. About NYMEX. https://www.investopedia.com/terms/n/nymex.asp#:~:text=NYMEX%20is%20a%20commodities%20trading,%2C%20energy%2C%20and%20agricultural%20commodities.
* [8] Wikipedia. About Wikipedia. https://en.wikipedia.org/wiki/TXU_Energy#:~:text=In%201984%2C%20DP%26L%2C%20TESCO%2C,Australia%2C%20Europe%20and%20North%20America.

### <b>ChatGPT</b>
* [1]
  * Purpose: To plot the frequency of messages per month to a line chart
  * Prompt: how can I use pandas package in python to plot the frequency of messages per month in a year, given the messages data are in a dataframe.
  * Note: A couple of tweaks were adjusted to the prompt, in order to get a correct response. Like for example the total number of messages must be sorted by year.
* [2]
  * Purpose: To find which employees have sent the most emails and plot that data to a bar chart.
  * Prompt: Hi ChatGPT, you are awesome! I am currently using Pandas package in Python and I want to know the top senders of email. I have a dataframe called 'messages_df' that contains the columns ['sender', 'subject']. The 'sender' column contains each employee's email address. I have another dataframe called 'employees_df' that contains the columns ['firstName', 'lastName', 'Email_id', 'Email2', 'Email3', 'EMail4']. There is in fact a typo in 'EMail4' where 'M' is capital but keep in mind that column is still spelt 'EMail4'. Now all of the email columns contain all employee's email address. Given information of messages_df and employees_df, I want to join them together to create a new dataframe based on messages_df's sender column to either any employees_df's 'Email_id'/'Email2'/'Email3'/'EMail4'.
* [3]
  * Purpose: To plot the total number of messages per month in the year 2001
  * Prompt: I am using pandas and matplotlib packages in Python. I have "messages_df" which is a dataframe that contains all messages. I want to plot the total number of messages per month in the year 2001.
  * Prompt Tweaks:
    * I want to show the name of the month instead of the number of the month and sort the months chronologically.